In [97]:
import time
import os
from selenium import webdriver
import json
import csv
import tkinter as tk
from PIL import ImageTk, Image

location = str(os.getcwd())

In [98]:
coding_list = []
with open(location + "/" + "minority_potentia_reliability.csv") as infile:
    reader = csv.reader(infile)
    author_list = list(reader)
list_item = 1
driver = None

name = (str(author_list[1][1])+" "+str(author_list[1][0]))
title = author_list[1][2]
institution = author_list[1][-3].split(",")[0]

In [99]:
#Initializing and Exiting
def initialize_browser_and_csv():
    global driver
    global list_item
    if not list_item:
        list_item = 1
    if not driver:
        options=webdriver.ChromeOptions()
        driver = webdriver.Chrome(location + "/chromedriver", options=options)
        url = 'https://google.com/'
        driver.get(url)

def exit_all():
    global driver
    global list_item
    global coding_list
    if list_item:
        list_item = None
    if driver:
        driver.close()
        driver = None
    if coding_list:
        with open((str(location) + '/author_coding.json'), 'w') as outfile:
            json.dump(coding_list, outfile)

#reults
def search_and_grab():
    global driver
    global name
    global title
    global institution
    search_string = ""
    
    search_method = search_method_var.get()
    if search_method == 1:
        search_string += ("https://scholar.google.com/scholar?hl=en&as_sdt=0%2C15&q=")
    elif search_method == 0:
        search_string += ("https://www.researchgate.net/search.Search.html?type=researcher&query=")
    else:
        search_string += ("https://www.google.com/search?q=")    
    search_string += (name)
    
    use_institute = institution_check_var.get()
    use_title = title_check_var.get()
    if use_institute == 1:
        search_string += ("+" + institution)
    if use_title == 1:
        search_string += ("+" + title)

    
    name = name
    driver.get(search_string)

    
#Moving through authors
def next_author():
    global name
    global title
    global institution
    global list_item
    if list_item:
        list_item+=1
    else:
        list_item=1
    try:
        if name:
            name = author_list[list_item][1] + " " + author_list[list_item][0]
            lname.config(text = ("Name: " + name))
        if title:
            title = (author_list[list_item][2])
            ltitle.config(text = ("Title: " + title))
        if institution:
            institution = author_list[list_item][-3].split(",")[0]
            linstitution.config(text = ("Institution: " + institution))
    except Exception as e:
        print(e)
        lname.config(text = "List Exhausted: Please Click Exit")
        ltitle.config(text = "")
        linstitution.config(text = '')

In [100]:
def submit_coding():
    global name
    global title
    global coding_list
    race = code_var.get()
    gender = gender_var.get()
    
    if race == 0:
        race_value = "Non-White"
    elif race == 1:
        race_value = "White"
    elif race == 2:
        race_value = "Unknown"
    else:
        race_value = ""
        
    if gender == 0:
        gender_value = "Female"
    elif gender == 1:
        gender_value = "Male"
    elif gender == 2:
        gender_value = "Binary"
    elif gender == 3:
        gender_value = "Unknown"
    else:
        gender_value = ""
    
    
    coding_list.append({"name": name, "title": title, "BIPOC": race_value, "gender": gender_value})
    next_author()

In [101]:
#GUI
window = tk.Tk()
institution_check_var = tk.IntVar()
title_check_var = tk.IntVar()
search_method_var = tk.IntVar()
code_var = tk.IntVar()
gender_var = tk.IntVar()
# window.geometry("1500x700")

#info
lname = tk.Label(text=("Name: " + str(author_list[1][1])+" "+str(author_list[1][0])), font=("Helvetica", 12))
lname.grid(row = 0, column = 0, sticky = "W", columnspan = 3)

linstitution = tk.Label(text=("Institution: " + author_list[1][-3].split(",")[0]), font=("Helvetica", 12))
linstitution.grid(row=1, column=0, columnspan = 3, sticky = "W")

ltitle = tk.Label(text=("Title: " + author_list[1][2]), font=("Helvetica", 12))
ltitle.grid(row = 2, column = 0, columnspan = 4, sticky = "W")

#Initialize and exit
start_button = tk.Button(text="Initialize", command = initialize_browser_and_csv, font=("Helvetica", 10))
start_button.grid(row = 0, column = 3)

exit_button = tk.Button(text="Quit/Compile", command=exit_all, font=("Helvetica", 10))
exit_button.grid(row = 1, column = 3)

#search terms
checkbox_1 = tk.Checkbutton(text="title", variable=title_check_var)
checkbox_1.grid(row = 4, column = 1)

checkbox_2 = tk.Checkbutton(text="institution", variable=institution_check_var)
checkbox_2.grid(row=3, column = 1)

#Search_method
research_gate_radio = tk.Radiobutton(text="Research Gate", variable=search_method_var, value=0)
research_gate_radio.grid(row=4, column = 0)

google_scholar_radio = tk.Radiobutton(text="Google Scholar", variable=search_method_var, value=1)
google_scholar_radio.grid(row=5, column = 0)

google_radio = tk.Radiobutton(text="Google", variable=search_method_var, value=2)
google_radio.grid(row=3, column = 0)

#go
search_button = tk.Button(text="Search", command = search_and_grab, font=("Helvetica", 10))
search_button.grid(row = 7, column = 1)

#Coding
bipoc = tk.Radiobutton(text="Mark as BIPOC", variable = code_var, value = 0, font=("Helvetica", 10))
bipoc.grid(row = 3, column = 3)

not_bipoc = tk.Radiobutton(text="Mark as not BIPOC", variable = code_var, value = 1, font=("Helvetica", 10))
not_bipoc.grid(row = 4, column = 3)

unknown_button = tk.Radiobutton(text="Mark as unknown", variable = code_var, value = 2, font=("Helvetica", 10))
unknown_button.grid(row = 5, column = 3)

skip = tk.Radiobutton(text="Racial Info Unavailable", variable = code_var, value = 3, font=("Helvetica", 10))
skip.grid(row = 6, column = 3)

#gender coding
female_radio = tk.Radiobutton(text="Female", variable = gender_var, value = 0, font=("Helvetica", 10))
female_radio.grid(row = 3, column = 2)

male_radio = tk.Radiobutton(text="Male", variable = gender_var, value = 1, font=("Helvetica", 10))
male_radio.grid(row = 4, column = 2)

nonbinary_radio = tk.Radiobutton(text="Non-Binary", variable = gender_var, value = 2, font=("Helvetica", 10))
nonbinary_radio.grid(row = 5, column = 2)

unknown_gender_radio = tk.Radiobutton(text="Gender Unknown", variable = gender_var, value = 3, font=("Helvetica", 10))
unknown_gender_radio.grid(row = 6, column = 2)

skip_gender_radio = tk.Radiobutton(text="Gender Info Unavailable", variable = gender_var, value = 4, font=("Helvetica", 10))
skip_gender_radio.grid(row = 7, column = 2)

submit = tk.Button(text = "Submit", command = submit_coding, font=("Helvetica", 10))
submit.grid(row = 7, column = 3)

window.mainloop()

In [50]:
# to do
# Accept input of an author and affiliation
# I think it would be good if apparent ethnicity is able to be further defined (African descent, Asian descent, Pacific Island descent, etc.)
# Outputs to an excel file that can be easily copy/pasted into a larger data base.